In [ ]:
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a random sample dataframe
#df = pd.DataFrame(np.random.randn(50, 4), columns=list('ABCD'))

# create a link to download the dataframe
#create_download_link(df)
# ↓ ↓ ↓  Yay, download link! ↓ ↓ ↓

In [ ]:
def print_unique_col_values(df):
    for column in df:
        #if df[column].dtypes=='object':
            print(f'{column}: {df[column].unique()}')

train = pd.read_csv("../input/turkiye-is-bankas-machine-learning-challenge-3/train.csv")

expenditures = pd.read_csv("../input/turkiye-is-bankas-machine-learning-challenge-3/monthly_expenditures.csv")

expenditures_copy=expenditures.copy()

expenditures_copy = expenditures_copy.pivot_table(index=['musteri'], 
            columns=['sektor','tarih'], values=['islem_adedi']).fillna(1).reset_index()

expenditures = expenditures.pivot_table(index=['musteri'], 
            columns=['sektor','tarih'], values=['aylik_toplam_tutar']).fillna(0).reset_index()
               
df2 = pd.merge(train, expenditures, "left", on = "musteri")
df2.drop(columns=['tarih','meslek_grubu'],inplace=True)#'meslek_grubu'
df2 = pd.merge(df2, expenditures_copy, "left", on = "musteri")

#fill NANs with the most frequent value
#df2 = df2.apply(lambda x:x.fillna(x.value_counts().index[0]))
#clip negative values in kidem_suresi
df2.loc[df2['kidem_suresi'] < 0, 'kidem_suresi'] = 0

#df2=pd.get_dummies(df2, columns=['egitim','is_durumu','meslek_grubu'],dummy_na=True,drop_first=True)#'meslek_grubu'

#from sklearn.preprocessing import LabelEncoder
#le=LabelEncoder()
#df2.egitim=le.fit_transform(df2.egitim)
#df2.is_durumu=le.fit_transform(df2.is_durumu)
#df2.meslek_grubu=le.fit_transform(df2.meslek_grubu)#remove this line


df2.drop(columns=['musteri'],inplace=True)#,'egitim_2eb5ddd72c','is_durumu_3773727d6e'

#columns_to_scale=list(df2.loc[ : , df2.columns != 'target'] )
#columns_to_scale=['yas','kidem_suresi']+(list(df2)[5:161])
#from sklearn.preprocessing import MinMaxScaler
#scaler=MinMaxScaler()
#df2[columns_to_scale]=scaler.fit_transform(df2[columns_to_scale])

df2=df2.dropna()
df2

In [ ]:
x=list(df2)[5:161]
list1=x[0:78]
list2=x[78:156]
for i in range(len(list1)):
    df2[list1[i]]=df2[list1[i]]/df2[list2[i]]

'''holder=0
count=0
while holder<len(list1):
    row=0
    for val in df2[list1[holder]]:
        if val!=0:
            val=val/(df2.iloc[row][list2[holder]])
            row+=1
        else:
            val=0
            row+=1
    #df2.drop(list1[i],inplace=True)
    #df2.drop(list2[holder],inplace=True)
    holder+=1
    count+=1
print(count)'''

In [ ]:
#df2=pd.get_dummies(df2, columns=['egitim','is_durumu','meslek_grubu'],dummy_na=True,drop_first=True)#'meslek_grubu'
#df2

In [ ]:
df2.drop(df2.iloc[:, 83:161], inplace = True, axis = 1) 
df2

In [ ]:
df2.drop(columns=['kidem_suresi'],inplace=True)


from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df2.egitim=le.fit_transform(df2.egitim)
df2.is_durumu=le.fit_transform(df2.is_durumu)

columns_to_scale=list(df2)[0:3]+list(df2)[4:82]
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
df2[columns_to_scale]=scaler.fit_transform(df2[columns_to_scale])
df2

In [ ]:
df2_columns=[]
test_df2_columns=[]
for col in df2:
    if col!='target':
        df2_columns.append(col)
for col in test_df2:
    test_df2_columns.append(col)
#print(df2_columns)
#print(test_df2_columns)

holder=0
count=0
while holder<len(df2_columns):
    #
    if df2_columns[holder]!=test_df2_columns[holder]:
        count+=1
        print(count)
        print(test_df2_columns[holder])
    holder+=1

In [ ]:
#check if there is missing values
#missing=df2[df2.isnull().any(axis=1)]
#print(missing)
print_unique_col_values(df2)
#print(df2.target.value_counts())

In [ ]:
#X = df2.drop('target',axis='columns')
#y = df2.target

from sklearn.model_selection import train_test_split
train, test = train_test_split(df2,test_size=0.2,random_state=42)

In [ ]:
df2.target.value_counts()

In [ ]:
#handling the imbalanced dataset by UPSAMPLING THE MINORITY CLASS

from sklearn.utils import resample
# Separate majority and minority classes
df_majority = train[train.target==0]
df_minority = train[train.target==1]

# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=45827,    # to match majority class
                                 random_state=123) # reproducible results

# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
train=df_upsampled

# Display new class counts
df_upsampled.target.value_counts()
# 1    576
# 0    576
# Name: balance, dtype: int64

In [ ]:
#handling the imbalanced dataset by DOWNSAMPLING THE MAJORITY CLASS

from sklearn.utils import resample
# Separate majority and minority classes
df_majority = train[train.target==0]
df_minority = train[train.target==1]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=1929,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
train=df_downsampled
 
# Display new class counts
df_downsampled.target.value_counts()
# 1    49
# 0    49
# Name: balance, dtype: int64

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from pylab import rcParams
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
%matplotlib inline
np.random.seed(27)
rcParams['figure.figsize'] = 10, 6
warnings.filterwarnings('ignore')
sns.set(style="darkgrid")

In [ ]:
def generate_model_report(y_actual, y_predicted):
    print("Accuracy = " , accuracy_score(y_actual, y_predicted))
    print("Precision = " ,precision_score(y_actual, y_predicted))
    print("Recall = " ,recall_score(y_actual, y_predicted))
    print("F1 Score = " ,f1_score(y_actual, y_predicted))
    pass

In [ ]:
def generate_auc_roc_curve(clf, X_test):
    y_pred_proba = clf.predict(X_test)#[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test,  y_pred_proba)
    auc = roc_auc_score(y_test, y_pred_proba)
    plt.plot(fpr,tpr,label="AUC ROC Curve with Area Under the curve ="+str(auc))
    plt.legend(loc=4)
    plt.show()
    pass

In [ ]:
target = 'target'
X = df2.loc[:, df2.columns!=target]
y = df2.loc[:, df2.columns==target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#ax = sns.countplot(x=target, data=df2)
#print(df2[target].value_counts())

In [ ]:
print(100* (2431/float(df2.shape[0])))
print(100* (57264/float(df2.shape[0])))

In [ ]:
from sklearn.svm import SVC

svm = SVC(random_state = 1,class_weight={0:0.05,1:0.95})

svm.fit(X_train,y_train)

print('Support Vector Machine Score : ',svm.score(X_test,y_test))

#from sklearn import svm
#from sklearn.svm import SVC
#clf = svm.SVC()
#clf.fit(X_train, y_train)
#SVC()

In [ ]:
#clf = RandomForestClassifier().fit(X_train, y_train)
y_Test_Pred = svm.predict(X_test)
pd.crosstab(y_Test_Pred, y_test[target], rownames=['Predicted'], colnames=['Actual'])

In [ ]:
generate_model_report(y_test, y_Test_Pred)
generate_auc_roc_curve(clf, X_test)

# class_weight='balanced'

In [ ]:
unique_classes = list(df2['target'].unique())
print(unique_classes)
out_dict = {}
for classes in unique_classes:
    out_dict[classes] = df2.shape[0]/((df2.loc[df2['target'] == classes].shape[0])
                                     *len(unique_classes))
print(out_dict)
print (X_train.shape, y_train.shape)

In [ ]:
clf = LogisticRegression(class_weight='balanced').fit(X_train, y_train)
from sklearn.utils import class_weight
class_weight.compute_class_weight('balanced', np.unique(y_train), y_train[target])
y_Test_Pred = clf.predict(X_test)
pd.crosstab(y_Test_Pred, y_test[target], rownames=['Predicted'], colnames=['Actual'])

In [ ]:
generate_model_report(y_test, y_Test_Pred)
generate_auc_roc_curve(clf, X_test)

In [ ]:
weights = np.linspace(0.05, 0.95, 20)
gsc = GridSearchCV(
    estimator=model(),
    param_grid={
        'class_weight': [{0: x, 1: 1.0-x} for x in weights]
    },
    scoring='f1',
    cv=5
)

grid_result = gsc.fit(X_train, y_train)
print("Best parameters : %s" % grid_result.best_params_)

In [ ]:
data_out = pd.DataFrame({'score': grid_result.cv_results_['mean_test_score'],
                       'weight': weights })
data_out.plot(x='weight')
data_out

In [ ]:
clf = RandomForestClassifier(**grid_result.best_params_).fit(X_train, y_train)
y_Test_Pred = clf.predict(X_test)
pd.crosstab(y_Test_Pred, y_test[target], rownames=['Predicted'], colnames=['Actual'])

In [ ]:
generate_model_report(y_test, y_Test_Pred)
generate_auc_roc_curve(clf, X_test)

# SMOTE

In [ ]:
unique, count = np.unique(y_train, return_counts=True)
y_train_dict_value_count = { k:v for (k,v) in zip(unique, count)}
y_train_dict_value_count

In [ ]:
sm = SMOTE(random_state=42, sampling_strategy=1.0)
X_train, y_train = sm.fit_sample(X_train, y_train)
unique, count = np.unique(y_train, return_counts=True)
y_train_smote_value_count = { k:v for (k,v) in zip(unique, count)}
y_train_smote_value_count

In [ ]:
clf = LogisticRegression().fit(X_train, y_train)
y_Test_Pred = clf.predict(X_test)
pd.crosstab(y_Test_Pred, y_test[target], rownames=['Predicted'], colnames=['Actual'])

In [ ]:
generate_model_report(y_test, y_Test_Pred)
generate_auc_roc_curve(clf, X_test)

In [ ]:
weights = np.linspace(0.005, 0.25, 10)
weights

In [ ]:
pipe = make_pipeline(
    SMOTE(),
    LogisticRegression()
)

weights = np.linspace(0.5, 0.75, 10)

gsc = GridSearchCV(
    estimator=pipe,
    param_grid={
        'smote__sampling_strategy': weights
    },
    scoring='f1',
    cv=3
)
grid_result = gsc.fit(X_train, y_train)

print("Best parameters : %s" % grid_result.best_params_)
weight_f1_score_df = pd.DataFrame({ 'score': grid_result.cv_results_['mean_test_score'],
                                   'weight': weights })
weight_f1_score_df.plot(x='weight')

In [ ]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

In [ ]:
pipe = make_pipeline(
    SMOTE(sampling_strategy=0.25),
    LogisticRegression()
)

pipe.fit(X_train, y_train) 
y_Test_Pred = (pipe.predict_proba(X_test)[:,1]>=0.25).astype(int) 
pd.crosstab(y_Test_Pred, y_test[target], rownames=['Predicted'], colnames=['Actual'])

In [ ]:
generate_model_report(y_test, y_Test_Pred)
generate_auc_roc_curve(clf, X_test)

###############################################################

In [ ]:
#preparing dataset
#X = df2.drop('target',axis='columns')
#y = df2['target']

#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10,stratify=y)

# Feature Scaling
#from sklearn.preprocessing import StandardScaler

#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

#training the random forest classfier
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(random_state=42,n_estimators=100,class_weight='balanced').fit(X_train, y_train)
#y_pred = classifier.predict_proba(X_test)
y_pred_01=classifier.predict(X_test)
#evaluation
from sklearn.metrics import roc_auc_score,accuracy_score,confusion_matrix,classification_report
print('auc_roc_score = ',roc_auc_score(y_test, y_pred_01))
print('accuracy = ',accuracy_score(y_test, y_pred_01))
print(classification_report(y_test, y_pred_01))
confusion_matrix(y_test, y_pred_01)

In [ ]:
test = pd.read_csv("../input/turkiye-is-bankas-machine-learning-challenge-3/test.csv")

test_expenditures = pd.read_csv("../input/turkiye-is-bankas-machine-learning-challenge-3/monthly_expenditures.csv")
#test_expenditures = test_expenditures.drop(columns=['islem_adedi'])
test_expenditures = test_expenditures.pivot_table(index=['musteri'], 
            columns=['sektor','tarih'], values='aylik_toplam_tutar','islem_adedi').fillna(0).reset_index()
                
test_df2 = pd.merge(test, test_expenditures, "left", on = "musteri")
test_df2.drop(columns=['tarih','meslek_grubu'],inplace=True)#'meslek_grubu'
#fill NANs with the most frequent value
test_df2 = test_df2.apply(lambda x:x.fillna(x.value_counts().index[0]))
#clip negative values in kidem_suresi
test_df2.loc[test_df2['kidem_suresi'] < 0, 'kidem_suresi'] = 0

from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
test_df2.egitim=le.fit_transform(test_df2.egitim)
test_df2.is_durumu=le.fit_transform(test_df2.is_durumu)
#test_df2.meslek_grubu=le.fit_transform(test_df2.meslek_grubu)#remove this line

test_musteri_column=test_df2.pop('musteri')
#test_df2.drop(columns=['musteri'],inplace=True)

#columns_to_scale=list(test_df2.loc[ : , test_df2.columns != 'target'] )
#from sklearn.preprocessing import MinMaxScaler
#scaler=MinMaxScaler()
#test_df2[columns_to_scale]=scaler.fit_transform(test_df2[columns_to_scale])

#test_df2

In [ ]:
test_df2

In [ ]:
yp = clf.predict(test_df2)
yp=yp.tolist()
yp

In [ ]:
y_pred = []
for element in yp[:,1]:
    if element > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [ ]:
print('0: ',yp.count(0),'\n1: ',yp.count(1))
# class 1 count is over 16000

In [ ]:
import csv

with open('result.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(zip(test_musteri_column, yp))
    
submission = pd.read_csv("./result.csv",names=['musteri','target'])
submission=submission.groupby('musteri').target.agg(lambda x : x.mode()[0]).reset_index()
#*******************************************#
submission.set_index('musteri').to_csv('LogisticRegression_DOWNSAMPLED_without_meslek_grubu.csv')
submission

#  **NOW WE WILL BUILD A DEEP LEARNING MODEL**

In [ ]:
X_train = train.drop('target',axis='columns')
y_train = train['target']
X_test=test.drop('target',axis='columns')
y_test = test['target']

#from imblearn.under_sampling import NearMiss
# Implementing Undersampling for Handling Imbalanced 
#nm = NearMiss()
#X_train,y_train=nm.fit_sample(X_train,y_train)
#X_train.shape,y_train.shape
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [ ]:
y_train = y_train['target']
y_test = y_test['target']

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, len(X_train.columns))

In [ ]:
import tensorflow as tf
from tensorflow import keras


model = keras.Sequential([
    keras.layers.Dense(64, input_shape=(81,), activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid')
])
'''
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(81,)),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])
#opt = keras.optimizers.Adam(learning_rate=0.01)
'''
#keras.optimizers.Adam(lr=1e-3)
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC()])

model.fit(X_train, y_train, class_weight={0:1,1:23.56},epochs=50)

In [ ]:
# class weighted neural network on an imbalanced classification dataset
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score
from keras.layers import Dense
from keras.models import Sequential

# prepare train and test dataset
#def prepare_data():
# generate 2d classification dataset
#X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0,
#n_clusters_per_class=2, weights=[0.99], flip_y=0, random_state=4)
# split into train and test
#n_train = 5000
#trainX, testX = X[:n_train, :], X[n_train:, :]
#trainy, testy = y[:n_train], y[n_train:]
#return trainX, trainy, testX, testy

# define the neural network model
def define_model(n_input):
    # define model
    model = Sequential()
    # define first hidden layer and visible layer
    model.add(Dense(10, input_dim=n_input, activation='relu', kernel_initializer='he_uniform'))
    # define output layer
    model.add(Dense(1, activation='sigmoid'))
    # define loss and optimizer
    model.compile(loss='binary_crossentropy', optimizer='Adam')
    return model

# prepare dataset
#trainX, trainy, testX, testy = prepare_data()
# get the model
n_input = X_train.shape[1]
model = define_model(n_input)
# fit model
weights = {0:1, 1:23.56}
history = model.fit(X_train, y_train, class_weight=weights, epochs=50, verbose=0)
# evaluate model
yhat = model.predict(X_test)
score = roc_auc_score(y_test, yhat)
print('ROC AUC: %.3f' % score)

In [ ]:
yp=model.predict_proba(X_test)
y_pred = []
for element in yp:
    if element > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)
y_pred=pd.Series(y_pred)
#y_pred

In [ ]:
pd.crosstab(y_pred, y_test, rownames=['Predicted'], colnames=['Actual'])

In [ ]:
from sklearn.metrics import roc_auc_score,accuracy_score,confusion_matrix,classification_report
print(classification_report(y_test, y_pred))

In [ ]:
generate_model_report(y_test, y_pred)
generate_auc_roc_curve(model, X_test)

In [ ]:
accuracy=model.evaluate(X_test, y_test)

##############################################################

In [ ]:
test = pd.read_csv("../input/turkiye-is-bankas-machine-learning-challenge-3/test.csv")

test_expenditures = pd.read_csv("../input/turkiye-is-bankas-machine-learning-challenge-3/monthly_expenditures.csv")

test_expenditures_copy=test_expenditures.copy()
test_expenditures_copy = test_expenditures_copy.pivot_table(index=['musteri'], 
            columns=['sektor','tarih'], values=['islem_adedi']).fillna(1).reset_index()


test_expenditures = test_expenditures.pivot_table(index=['musteri'], 
            columns=['sektor','tarih'], values=['aylik_toplam_tutar']).fillna(0).reset_index()
                
test_df2 = pd.merge(test, test_expenditures, "left", on = "musteri")
test_df2.drop(columns=['tarih','meslek_grubu'],inplace=True)#'meslek_grubu'

test_df2 = pd.merge(test_df2, test_expenditures_copy, "left", on = "musteri")

#fill NANs with the most frequent value
test_df2 = test_df2.apply(lambda x:x.fillna(x.value_counts().index[0]))
#clip negative values in kidem_suresi
test_df2.loc[test_df2['kidem_suresi'] < 0, 'kidem_suresi'] = 0

#test_df2=pd.get_dummies(test_df2, columns=['egitim','is_durumu',],dummy_na=True,drop_first=True)#'meslek_grubu'

#from sklearn.preprocessing import LabelEncoder
#le=LabelEncoder()
#test_df2.egitim=le.fit_transform(test_df2.egitim)
#test_df2.is_durumu=le.fit_transform(test_df2.is_durumu)
#test_df2.meslek_grubu=le.fit_transform(test_df2.meslek_grubu)#remove this line

test_musteri_column=test_df2.pop('musteri')
#test_df2.drop(columns=['musteri'],inplace=True)

#test_df2.insert(158,'egitim_2eb5ddd72c',0)
#test_df2.insert(164,'is_durumu_3773727d6e',0)

#columns_to_scale=list(test_df2.loc[ : , test_df2.columns != 'target'] )
#from sklearn.preprocessing import MinMaxScaler
#scaler=MinMaxScaler()
#test_df2[columns_to_scale]=scaler.fit_transform(test_df2[columns_to_scale])
#test_df2.dropna(inplace=True)
test_df2

In [ ]:
x=list(test_df2)[4:160]
list1=x[0:78]
list2=x[78:156]
for i in range(len(list1)):
    test_df2[list1[i]]=test_df2[list1[i]]/test_df2[list2[i]]

'''holder=0
count=0
while holder<len(list1):
    row=0
    for val in test_df2[list1[holder]]:
        if val!=0:
            val=val/(test_df2.iloc[row][list2[holder]])
            row+=1
        else:
            val=0
            row+=1
    #df2.drop(list1[i],inplace=True)
    #df2.drop(list2[holder],inplace=True)
    holder+=1
    count+=1
print(count)'''

In [ ]:
#test_df2=pd.get_dummies(test_df2, columns=['egitim','is_durumu','meslek_grubu'],dummy_na=True,drop_first=True)#'meslek_grubu'
test_df2.drop(test_df2.iloc[:, 82:160], inplace = True, axis = 1)

#test_df2.insert(80,'egitim_2eb5ddd72c',0)
#test_df2.insert(86,'is_durumu_3773727d6e',0)

test_df2

In [ ]:
test_df2.drop(columns=['kidem_suresi'],inplace=True)
columns_to_scale=list(test_df2)[0:3]+list(test_df2)[3:81]

from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
test_df2.egitim=le.fit_transform(test_df2.egitim)
test_df2.is_durumu=le.fit_transform(test_df2.is_durumu)

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
test_df2[columns_to_scale]=scaler.fit_transform(test_df2[columns_to_scale])
test_df2

In [ ]:
yp = model.predict(test_df2)
#yp

In [ ]:
type(y_pred)

In [ ]:
y_pred = []
for element in yp:
    if element > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)
type(y_pred)

In [ ]:
list_of_zeros=y_pred

In [ ]:
list_of_ones=y_pred

In [ ]:
final_list=[]
for i in range(40000):
    if list_of_zeros[i]==0 and list_of_ones[i]==0:
        final_list.append(list_of_zeros[i])
    elif list_of_zeros[i]==0 and list_of_ones[i]==1:
        final_list.append(list_of_zeros[i])
    elif list_of_zeros[i]==1 and list_of_ones[i]==0:
        final_list.append(list_of_ones[i])
    else:
        final_list.append(list_of_ones[i])

In [ ]:
final_list.count(0)

In [ ]:
class_count=str(' class0: '+str(y_pred.count(0))+'  class1: '+str(y_pred.count(1)))
print(class_count)

In [ ]:
y_pred=pd.Series(y_pred)
type(y_pred)

In [ ]:
sample= str('sample(droupout 0.5)= ' +str(int(len(df2.target)/2))+class_count+' accuracy= '+str(accuracy[1]))
list_of_samples.append(sample)
list_of_samples

In [ ]:
type(test_musteri_column)

In [ ]:
import csv

with open('result.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(zip(test_musteri_column, y_pred))
    
submission = pd.read_csv("./result.csv",names=['musteri','target'])
#submission=submission.groupby('musteri').target.agg(lambda x : x.mode()[0]).reset_index()
#*******************************************#
submission.set_index('musteri').to_csv('ANN_class_weighted.csv')
submission

In [ ]:
list_of_samples=['sample= 3000 class0: 22591  class1: 17409 accuracy= 0.7513161301612854',
 'sample= 4000 class0: 22434  class1: 17566 accuracy= 0.751573920249939',
 'sample= 5000 class0: 22349  class1: 17651 accuracy= 0.7583276033401489']